# Ensemble Learning

## Initial Imports

In [126]:
import warnings
warnings.filterwarnings('ignore')

In [127]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [128]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced 
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Read the CSV and Perform Basic Data Cleaning

In [129]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500,0.1719,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0,0,65687,38199,2000,61987,N,N
1,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1,0,271427,60641,41200,49197,N,N
2,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0,0,60644,45684,7500,43144,N,N
3,10000,0.1640,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1,0,99506,68784,19700,76506,N,N
4,22000,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0,0,219750,25919,27600,20000,N,N


## Split the Data into Training and Testing

In [130]:
# Create our features
columns=["loan_amnt","int_rate","installment","annual_inc"]
X = df[columns]
# Create our target
y = df["loan_status"]

In [154]:
# Check the balance of our target values
# YOUR CODE HERE
X.describe()

,loan_amnt,int_rate,installment,annual_inc
count,68817.000000,68817.000000,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04
std,10277.348590,0.048130,288.062432,1.155800e+05
min,1000.000000,0.060000,30.890000,4.000000e+01
25%,9000.000000,0.088100,265.730000,5.000000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05
max,40000.000000,0.308400,1676.230000,8.797500e+06


In [155]:
y.value_counts

<bound method IndexOpsMixin.value_counts of 0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object>

In [132]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 4)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [133]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [134]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [156]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [157]:
# Resample the training data with the RandomForestClassifier
# YOUR CODE HERE
rf_model = RandomForestClassifier(n_estimators=50, random_state=7)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [158]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# predictions = rf_model.predict(X_test_scaled)
X_test_scaled = X_scaler.transform(X_test)
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test,predictions)
print(acc_score)

0.9938390002906132


In [159]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [160]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00        87
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17118

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [161]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(importances, X.columns), reverse=True)
importances_sorted[:10]

[(0.5149128776128726, 'annual_inc'),
 (0.2510169167599696, 'installment'),
 (0.1294210938813998, 'loan_amnt'),
 (0.10464911174575807, 'int_rate')]

### Easy Ensemble Classifier

In [162]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
rf_model = EasyEnsembleClassifier(n_estimators=50, random_state=7)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [163]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = accuracy_score(y_test,predictions)
print(acc_score)

0.9938390002906132


In [164]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [165]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00        87
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17118

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

#### Easy Ensemble Classifier

2. Which model had the best recall score?

#### Both had same score

3. Which model had the best geometric mean score?
#### Both had same score

4. What are the top three features?

#### Annual income,  size of installment, loan amount